In [1]:
import torch

In [2]:
from torch import nn

In [5]:
a = torch.ones((30, 60, 40))

In [8]:
avg = nn.AvgPool1d(kernel_size = 2, stride = 2)

In [9]:
avg(a).shape

torch.Size([30, 60, 20])

In [10]:
nn.Conv1d??

In [14]:
conv = nn.Conv1d(60, 80, kernel_size = 1, stride = 1)

In [47]:
a = torch.ones((5, 50, 10))
b = torch.ones((5, 10, 1))

In [ ]:
(N, 1, T), (N, T, value_size)

In [69]:
a = torch.ones((10, 1, 20))
b = torch.ones((10, 20, 3))
torch.bmm(a, b).shape

torch.Size([10, 1, 3])

In [49]:
context = torch.ones((10, 10))

In [53]:
lengths = torch.tensor([1,2,3,4,5,6,7,8,9,10])

In [65]:
attention = torch.ones((10, 10))

In [56]:
# Create an (N, T) boolean mask for all padding positions
# Make use of broadcasting: (1, T), (N, 1) -> (N, T)
mask = torch.arange(context.size(1)).unsqueeze(0) >= lengths.unsqueeze(1)
# Set attention logits at padding positions to negative infinity.
attention.masked_fill_(mask, -1e9)
# Take softmax over the "source length" dimension.
attention = nn.functional.softmax(attention, dim=1)


In [60]:
torch.arange(context.size(1)).unsqueeze(0) >= lengths.unsqueeze(1)

tensor([[False,  True,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True,  True,  True,  True,  True],
        [False, False, False, False,  True,  True,  True,  True,  True,  True],
        [False, False, False, False, False,  True,  True,  True,  True,  True],
        [False, False, False, False, False, False,  True,  True,  True,  True],
        [False, False, False, False, False, False, False,  True,  True,  True],
        [False, False, False, False, False, False, False, False,  True,  True],
        [False, False, False, False, False, False, False, False, False,  True],
        [False, False, False, False, False, False, False, False, False, False]])

In [9]:
import os
import numpy 
from wsj_loader import WSJ

os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ['WSJ_PATH'] = './data'
loader = WSJ()
trainX, trainY = loader.train
devX, devY = loader.dev

In [11]:
import numpy as np

In [22]:
# speech_train = np.load('./data/train.npy', allow_pickle=True, encoding='bytes')
# speech_valid = np.load('dev.npy', allow_pickle=True, encoding='bytes')
# speech_test = np.load('test.npy', allow_pickle=True, encoding='bytes')

transcript_train = np.load('./data/train_transcripts.npy', allow_pickle=True,encoding='ASCII')
transcript_valid = np.load('./data/dev_transcripts.npy', allow_pickle=True,encoding='bytes')
print("Data Loading Sucessful.....")

In [31]:
letter_list = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q',\
             'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '-', "'", '.', '_', '+', ' ','<sos>','<eos>']

for utterance in transcript_train:
    lst = [byte.decode('utf8') for byte in utterance]
    break

# segset = np.array([l.decode(‘utf8’) for l in s for s in transcript_train])

In [36]:
from util import *

transform_letter_to_index(transcript_train, letter_list)

In [33]:
a = []
a.insert(0, '<es>')